In [18]:
import webbrowser
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from geopy.geocoders import Nominatim
import folium

path_shape_file = "../Data/ShapeFiles/States/tl_2022_us_state.shp"

# Read the Shapefile
gdf = gpd.read_file(path_shape_file)

# Convert to GeoJSON format
# gdf.to_file('output.geojson', driver='GeoJSON')

data = {
    'NAME': ['West Virginia', 'California', 'Texas', 'Florida'],
    'Population': [19530351, 39538223, 29145505, 21538187],
}


df_data = pd.DataFrame(data)

merged_data = gdf.merge(df_data, left_on='NAME', right_on='NAME', how='left')

# Create a map without specifying a center or zoom level
m = folium.Map([37.090240,-95.712891], zoom_start=4)

folium.GeoJson(
    'output.geojson',
    style_function=lambda feature: {
        'fillColor': 'green',  # You can customize the state colors
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.6,
        'popup': f"State: {feature['properties']['NAME']}<br>Population: {merged_data[merged_data['NAME'] == feature['properties']['NAME']]['Population'].values[0]}"
    }
).add_to(m)

m

True